# Logistic regression

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [ ]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

### Hypothesis

In [ ]:
from IPython.display import Image
image_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/99785e9d01523e8bb6bf78d1220c1b020fa6c7c8/image_day1/Hypo.png'
Image(image_url)

### cost

In [ ]:
from IPython.display import Image
image_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/99785e9d01523e8bb6bf78d1220c1b020fa6c7c8/image_day1/cost.png'
Image(image_url)

Pytorch has binary cross entropy function:  
torch.nn.functional.binary_cross_entropy(input, target, ...)


softmax와 label의 elemental wise 곱을하면 label이 0인 확률들은 모두 없어지게 되어 아래 수식이 가능  
따라서 class에는 one-hot encoding 되지 않은 값이 들어가야 함  
x에는 soft max를 거치지 않은 벡터가 들어감(CrossEntropyLoss()에 softmax가 포함)

In [ ]:
predict = torch.FloatTensor(np.array([[0.1], [0.2], [0.9]]))
label = torch.FloatTensor(np.array([[0], [0], [1]]))

In [ ]:
 F.binary_cross_entropy(predict, label)

## Training

In [ ]:
class my_logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid() # Just add sigmoid function!

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [ ]:
model = my_logistic()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} '.format(
            epoch, nb_epochs, cost.item(), 
        ))

### 결과 출력

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
x1 = torch.linspace(0, 10, 100).reshape(-1, 1)
x2 = torch.linspace(0, 10, 100).reshape(-1, 1)

In [ ]:
pca = PCA(n_components=1)
x_pca_100 = pca.fit_transform(torch.cat((x1, x2),axis=1))

In [ ]:
x_pca = pca.transform(x_train)

In [ ]:
hypothesis = model(torch.cat((x1, x2),axis=1))

In [ ]:
hx = hypothesis.detach().numpy()

In [ ]:
plt.scatter(x_pca, y_train, s=50)
plt.scatter(x_pca_100, hx, s=10)

### Accuracy

In [ ]:
hypothesis = model(x_train)

prediction = hypothesis >= torch.FloatTensor([0.5])

correct_prediction = prediction.float() == y_train

accuracy = correct_prediction.sum().item() / correct_prediction.shape[0]

accuracy * 100